In [14]:
import pandas as pd
import numpy as np
import json
import csv

In [2]:
#1.确定这几个数据都是干什么的
swiping_card_dataframe = pd.read_csv('./bus_swiping_card_info1.csv')
card_info_dataframe = pd.read_csv('./card_info.csv')
bus_swip_dataframe = pd.read_excel('公交数据.xlsx',sheet_name='刷卡数据',header=0,engine='openpyxl')
bus_station_dataframe = pd.read_excel('公交数据.xlsx',sheet_name='站点数据',header=0,engine='openpyxl')

In [3]:
#swiping_card_dataframe ： 每次刷卡的信息：几号线，第几站，车辆的id，刷卡的id，刷卡的时间
#card_info_dataframe ： 每次刷卡的信息：几号线，第几站，车辆的id，刷卡的id，刷卡的时间也是上面的信息，但是时间是去年的。
#用去年的时间的同比数据，去预测今年的同比数据，看是否一致，如果一致，说明经济发展没有变化，如果变大，说明经济好转
#bus_swip_dataframe ：一天的统计信息：各条线路、各个站点的刷卡数量信息
#bus_station_dataframe ： 站点的统计信息：各个站点的经纬度

In [56]:
line_station_number_sequen_dataframe = \
bus_station_dataframe.groupby(['line_identity','station_number']).agg({'station_name':list,'station_identity':list}).reset_index()
line_station_number_sequen_dataframe['length'] = line_station_number_sequen_dataframe['station_identity'].apply(lambda x:len(x))
line_station_number_sequen_dataframe

,line_identity,station_number,station_name,station_identity,length
0,1,20,"[火车站, 展览馆, 新路口, 邮电大楼, 大十字, 喷水池, 云中广场, 六广门, 北京路...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",20
1,2,20,"[火车站, 展览馆, 新路口, 河滨公园, 次南门, 大西门, 紫林庵, 延安西路, 黔灵山...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",20
2,3,29,"[贵州理工学院, 贵黄交警支队, 枫丹白鹭花园, 后坝, 三桥水果市场, 三桥, 三桥北路口...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",29
3,4,44,"[新寨公交枢纽, 南厂路南, 军区干休所, 武岳新村, 军区二招, 南厂路北, 纪念塔, 箭...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",44
4,5,36,"[甘荫塘轨道站, 甘荫塘, 四方河路站, 通银配件城, 皂角井站, 凤凰翠堤, 新发装饰市场...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",36
...,...,...,...,...,...
320,10050,46,"[一八三厂, 毛寨村, 小河平桥, 黄河路南, 黄河路北, 四十四医院, 新村, 四方河, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",46
321,10056,31,"[乐湾国际城, 东风镇, 贵州师范学院, 云上村, 117地质队, 温泉路口, 新添寨, 市...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",31
322,10229,28,"[金阳客站, 商城东路, 宾阳大道, 兴筑西路, 碧海乾图, 黔中一中南, 老阳关, 北京路...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",28
323,10262,24,"[黔中北站, 甲秀北路, 杨家庄, 大关, 鹿冲关公园, 相宝山, 师大, 省公安厅, 宝山...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",24


In [77]:
line_station_number_sequen_dataframe['id_station'] = \
line_station_number_sequen_dataframe.apply(lambda x:list(zip(x['station_identity'],x['station_name'])),axis=1)

In [81]:
line_station_number_sequen_dataframe

,line_identity,station_number,station_name,station_identity,length,id_station
0,1,20,"[火车站, 展览馆, 新路口, 邮电大楼, 大十字, 喷水池, 云中广场, 六广门, 北京路...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",20,"[(1, 火车站), (2, 展览馆), (3, 新路口), (4, 邮电大楼), (5, ..."
1,2,20,"[火车站, 展览馆, 新路口, 河滨公园, 次南门, 大西门, 紫林庵, 延安西路, 黔灵山...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",20,"[(1, 火车站), (2, 展览馆), (3, 新路口), (4, 河滨公园), (5, ..."
2,3,29,"[贵州理工学院, 贵黄交警支队, 枫丹白鹭花园, 后坝, 三桥水果市场, 三桥, 三桥北路口...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",29,"[(1, 贵州理工学院), (2, 贵黄交警支队), (3, 枫丹白鹭花园), (4, 后坝..."
3,4,44,"[新寨公交枢纽, 南厂路南, 军区干休所, 武岳新村, 军区二招, 南厂路北, 纪念塔, 箭...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",44,"[(1, 新寨公交枢纽), (2, 南厂路南), (3, 军区干休所), (4, 武岳新村)..."
4,5,36,"[甘荫塘轨道站, 甘荫塘, 四方河路站, 通银配件城, 皂角井站, 凤凰翠堤, 新发装饰市场...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",36,"[(1, 甘荫塘轨道站), (2, 甘荫塘), (3, 四方河路站), (4, 通银配件城)..."
...,...,...,...,...,...,...
320,10050,46,"[一八三厂, 毛寨村, 小河平桥, 黄河路南, 黄河路北, 四十四医院, 新村, 四方河, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",46,"[(1, 一八三厂), (2, 毛寨村), (3, 小河平桥), (4, 黄河路南), (5..."
321,10056,31,"[乐湾国际城, 东风镇, 贵州师范学院, 云上村, 117地质队, 温泉路口, 新添寨, 市...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",31,"[(1, 乐湾国际城), (2, 东风镇), (3, 贵州师范学院), (4, 云上村), ..."
322,10229,28,"[金阳客站, 商城东路, 宾阳大道, 兴筑西路, 碧海乾图, 黔中一中南, 老阳关, 北京路...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",28,"[(1, 金阳客站), (2, 商城东路), (3, 宾阳大道), (4, 兴筑西路), (..."
323,10262,24,"[黔中北站, 甲秀北路, 杨家庄, 大关, 鹿冲关公园, 相宝山, 师大, 省公安厅, 宝山...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",24,"[(1, 黔中北站), (2, 甲秀北路), (3, 杨家庄), (4, 大关), (5, ..."


In [85]:
#获取每条线路上乘客上车站所对应的可能的下车站：估计数据量为：
#构建一个字典：
#road_structure = {line_id:{'stations_nums':int,'stations':{station_id:station_name},'all_stations':[]}}
road_line_station_structure_setting={}
for row_index in range(len(line_station_number_sequen_dataframe)):
    line_id = line_station_number_sequen_dataframe[row_index:row_index+1]['line_identity'].values[0]
    stations_nums = line_station_number_sequen_dataframe[row_index:row_index+1]['station_number'].values[0]
    all_stations = line_station_number_sequen_dataframe[row_index:row_index+1]['station_name'].values[0]
    stations={}
    for instance in line_station_number_sequen_dataframe[row_index:row_index+1]['id_station'].values[0]:
        instance_id = instance[0]
        instance_name = instance[1]
        stations[instance_id]=instance_name
    road_line = {'stations_nums':stations_nums,'stations':stations,'all_stations':all_stations}
    road_line_station_structure_setting[line_id] = road_line    

In [86]:
road_line_station_structure_setting

{1: {'stations_nums': 20,
  'stations': {1: '火车站',
   2: '展览馆',
   3: '新路口',
   4: '邮电大楼',
   5: '大十字',
   6: '喷水池',
   7: '云中广场',
   8: '六广门',
   9: '北京路',
   10: '北京西路口',
   11: '黔灵山公园',
   12: '黔灵山公园',
   13: '延安西路',
   14: '紫林庵',
   15: '大西门',
   16: '次南门',
   17: '河滨公园',
   18: '新路口',
   19: '展览馆',
   20: '火车站'},
  'all_stations': ['火车站',
   '展览馆',
   '新路口',
   '邮电大楼',
   '大十字',
   '喷水池',
   '云中广场',
   '六广门',
   '北京路',
   '北京西路口',
   '黔灵山公园',
   '黔灵山公园',
   '延安西路',
   '紫林庵',
   '大西门',
   '次南门',
   '河滨公园',
   '新路口',
   '展览馆',
   '火车站']},
 2: {'stations_nums': 20,
  'stations': {1: '火车站',
   2: '展览馆',
   3: '新路口',
   4: '河滨公园',
   5: '次南门',
   6: '大西门',
   7: '紫林庵',
   8: '延安西路',
   9: '黔灵山公园',
   10: '黔灵山公园',
   11: '北京西路口',
   12: '北京路',
   13: '六广门',
   14: '云中广场',
   15: '喷水池',
   16: '大十字',
   17: '邮电大楼',
   18: '新路口',
   19: '展览馆',
   20: '火车站'},
  'all_stations': ['火车站',
   '展览馆',
   '新路口',
   '河滨公园',
   '次南门',
   '大西门',
   '紫林庵',
   '延安西路',
   '黔灵山公园',
   '黔灵山公园',
   '北京西路口'

In [12]:
station_distance_rank_index_list = np.loadtxt('./station_pair_distance_rank_index.csv',delimiter=',',dtype=int)

In [17]:
# 从文件中读取字典
with open('./id_2_station.json', 'r') as file:
    id_2_station = json.load(file)
# 将字典保存到文件中
with open('./station_2_id.json', 'r') as file:
    station_2_id = json.load(file)

In [20]:
#从card_info_dataframe中找出所有用户一天内的od对信息
card_info_dataframe['custom_day'] = card_info_dataframe['custom_time'].apply(lambda x:x.split(' ')[0])
card_info_dataframe['custom_precise_time'] = card_info_dataframe['custom_time'].apply(lambda x:x.split(' ')[1])
#sum(card_info_dataframe['partitionday']==card_info_dataframe['custom_day'])
#card_info_dataframe
card_info_dataframe.drop(labels=['custom_time','card_type','consume','data_src','data_load_time','partitionday'],axis=1,inplace=True)

In [171]:
card_info_dataframe

,card_identity,line_identity,car_identity,station_identity,custom_day,custom_precise_time
0,0x1FDD5BE63E904A3230A0AE9AA0E40C61,28,7520,54,2023-09-29,08:59:40
1,0x1FFC5D30D96AE2C4524842A09AA67C44,84,5085,24,2023-09-29,11:08:22
2,0x65F9DAD54040FBE48972B10F3B6337DC,25,6718,4,2023-09-29,08:51:28
3,0x747A26B4727A9A3094A1881F29D01510,208,6655,1,2023-09-29,22:32:48
4,0x81A8FFB4112FDBA4782A4FB50E1C0EE0,204,4472,46,2023-09-29,19:50:37
...,...,...,...,...,...,...
724622,0x6B5CCBC949D06967421DDEFFA8682697,13,7618,22,2023-10-01,11:09:08
724623,0x7FF2BF197AE5BB5ECCBC9BC64FFEE659,7,5020,10,2023-10-01,15:34:58
724624,0x89C88A5125FCAC7607C396EDABD8FA50,320,4451,9,2023-10-01,10:13:20
724625,0xA159FB0957C071A17B4FC41C6F622556,61,3826,10,2023-10-01,17:22:23


In [29]:
day_0929_dataframe = card_info_dataframe[card_info_dataframe['custom_day']=='2023-09-29']
day_0929_dataframe

,card_identity,line_identity,car_identity,station_identity,custom_day,custom_precise_time
0,0x1FDD5BE63E904A3230A0AE9AA0E40C61,28,7520,54,2023-09-29,08:59:40
1,0x1FFC5D30D96AE2C4524842A09AA67C44,84,5085,24,2023-09-29,11:08:22
2,0x65F9DAD54040FBE48972B10F3B6337DC,25,6718,4,2023-09-29,08:51:28
3,0x747A26B4727A9A3094A1881F29D01510,208,6655,1,2023-09-29,22:32:48
4,0x81A8FFB4112FDBA4782A4FB50E1C0EE0,204,4472,46,2023-09-29,19:50:37
...,...,...,...,...,...,...
377159,0x51A33F38105E35D006C03F0694252007,52,5054,22,2023-09-29,08:48:38
377160,0x5BD2D7FBE1BE24A24AFD4D3ECA27FD3F,1,4662,6,2023-09-29,10:33:45
377161,0x6D191CC173F9415C1148B434D2F9012E,203,4810,5,2023-09-29,14:57:10
377162,0x7896A07D162530E734F876D16301EFEB,73,5176,23,2023-09-29,15:08:49


In [68]:
#按照刷卡id进行聚合
every_user_all_trips_dataframe = \
day_0929_dataframe.groupby('card_identity').agg({'custom_precise_time':list, \
                                                 'line_identity':list, \
                                                 'station_identity':list, \
                                                'car_identity':list},axis=1).reset_index()
#每天承载18万人出行181190人
#乘坐两程以上的有109927人
#乘坐单程的极有可能是流动，也有可能是别的交通方式回。从前后相依有下一段旅程的来确定od对，对于没有后程的暂时不确定。
every_user_all_trips_dataframe['trip_length'] = every_user_all_trips_dataframe['line_identity'].apply(lambda x:len(x))
#every_user_all_trips_dataframe
trip_great_than_2=every_user_all_trips_dataframe[every_user_all_trips_dataframe['trip_length']!=1].reset_index(drop=True)
#trip_great_than_2
trip_great_than_2['trip'] = trip_great_than_2.apply(lambda x:list(zip(x['custom_precise_time'],x['line_identity'],x['station_identity'])),axis=1)
trip_great_than_2

,card_identity,custom_precise_time,line_identity,station_identity,car_identity,trip_length,trip
0,0x0000719AE3ECECBABAD0263AF940450F,"[19:01:59, 16:55:43]","[40, 46]","[31, 1]","[4483, 3762]",2,"[(19:01:59, 40, 31), (16:55:43, 46, 1)]"
1,0x0000D15C7814D19BE81C40B3ED9CA55C,"[09:26:05, 19:24:18, 16:29:50, 09:20:51]","[207, 202, 5, 204]","[14, 39, 5, 8]","[7111, 4622, 4786, 4400]",4,"[(09:26:05, 207, 14), (19:24:18, 202, 39), (16..."
2,0x000229AC214F69A81F266A521394C628,"[07:13:30, 18:07:56]","[7, 19]","[21, 23]","[5020, 5038]",2,"[(07:13:30, 7, 21), (18:07:56, 19, 23)]"
3,0x00065B8345E25E499BDF9B0632576C87,"[16:26:36, 09:58:48]","[57, 57]","[6, 32]","[7344, 7343]",2,"[(16:26:36, 57, 6), (09:58:48, 57, 32)]"
4,0x00067C25B50FB204998F9955D5692DCD,"[13:42:06, 11:24:18]","[16, 306]","[22, 14]","[4536, 5181]",2,"[(13:42:06, 16, 22), (11:24:18, 306, 14)]"
...,...,...,...,...,...,...,...
109922,0xFFFD09DB2DE6FD874710FA9B56E9A1B4,"[09:42:40, 08:03:14]","[8, 319]","[22, 15]","[4875, 4440]",2,"[(09:42:40, 8, 22), (08:03:14, 319, 15)]"
109923,0xFFFD5320ADB5DC148D136F1494654247,"[12:46:31, 12:16:55]","[35, 9]","[8, 2]","[4881, 3882]",2,"[(12:46:31, 35, 8), (12:16:55, 9, 2)]"
109924,0xFFFDE3BEFCD5F71FFAC2FE888410D3BC,"[16:00:55, 16:01:00, 15:07:13, 15:07:03]","[12, 12, 229, 229]","[14, 14, 34, 34]","[7303, 7303, 7151, 7151]",4,"[(16:00:55, 12, 14), (16:01:00, 12, 14), (15:0..."
109925,0xFFFDF917CDD67471BC901AD187061D38,"[14:03:34, 13:16:22]","[47, 60]","[2, 6]","[5042, 4484]",2,"[(14:03:34, 47, 2), (13:16:22, 60, 6)]"


In [93]:
can_used_od_trips=[]
def find_leave_station(sorted_index_array:np.ndarray,specified_indices:list):
    # 找到指定索引组中在排序数组中的位置
    positions = [sorted_index_array.index(idx) for idx in specified_indices]
    return sorted_index_array[min(positions)]
for row_index in range(len(trip_great_than_2)):
    trip_list = trip_great_than_2[row_index:row_index+1]['trip'].values[0]
    sorted_trips_list = sorted(trip_list,key=lambda x:x[0],reverse=False)
    for trip_index in range(len(sorted_trips_list)-1):
        this_time,this_line,this_station_id = sorted_trips_list[trip_index]
        next_time,next_line,next_station_id = sorted_trips_list[trip_index+1]
        try:
            this_line_station_name = road_line_station_structure_setting[this_line]['stations'][this_station_id]
            this_line_station_id = station_2_id[this_line_station_name]
        
            all_possible_leave_stations = road_line_station_structure_setting[this_line]['all_stations']
            all_possible_leave_stations_index = [station_2_id[station] for station in all_possible_leave_stations]
        
            next_line_station_name = road_line_station_structure_setting[next_line]['stations'][next_station_id]
            next_line_station_id = station_2_id[next_line_station_name]
            distance_2_next_line_station_ndarray  = station_distance_rank_index_list[next_line_station_id]
            leave_station = find_leave_station(distance_2_next_line_station_ndarray.tolist(),all_possible_leave_stations_index)
            od_trip = (this_line_station_id,this_time,this_line,next_line_station_id)
            can_used_od_trips.append(od_trip)
        except:
            continue

In [103]:
#将可用的od对转化为dataframe：这样就可以使用dataframe进行聚类
#can_used_od_trips[0:5]
can_used_od_trips_dataframe = pd.DataFrame(columns=['start_station_id','time','line','end_station_id'])
for trip_index in range(len(can_used_od_trips)):
    start_station_id,time,line,end_station_id=can_used_od_trips[trip_index]
    can_used_od_trips_dataframe.loc[len(can_used_od_trips_dataframe)] = \
    {'start_station_id':start_station_id, \
                                         'time':time, \
                                         'line':line, \
                                         'end_station_id':end_station_id}

In [104]:
len(can_used_od_trips)
#一共有169099可以确定的od对数据

169099

In [107]:
can_used_od_trips_dataframe['hour']= \
can_used_od_trips_dataframe['time'].apply(lambda x:x.split(':')[0])
can_used_od_trips_dataframe['minu']= \
can_used_od_trips_dataframe['time'].apply(lambda x:x.split(':')[1])

In [105]:
can_used_od_trips_dataframe

,start_station_id,time,line,end_station_id
0,1435,16:55:43,46,1262
1,643,09:20:51,204,436
2,436,09:26:05,207,898
3,898,16:29:50,5,406
4,795,07:13:30,7,1416
...,...,...,...,...
169094,1155,15:07:03,229,1155
169095,1155,15:07:13,229,263
169096,263,16:00:55,12,263
169097,1234,13:16:22,60,419


In [124]:
can_used_od_trips_dataframe.to_json('./can_used_od_trips_dataframe.json', orient='records')

In [125]:
# 使用read_json()函数打开JSON文件
test_can_used_od_trips_dataframe = pd.read_json('./can_used_od_trips_dataframe.json', orient='records')

In [126]:
test_can_used_od_trips_dataframe

,start_station_id,time,line,end_station_id,hour,minu
0,1435,16:55:43,46,1262,16,55
1,643,09:20:51,204,436,9,20
2,436,09:26:05,207,898,9,26
3,898,16:29:50,5,406,16,29
4,795,07:13:30,7,1416,7,13
...,...,...,...,...,...,...
169094,1155,15:07:03,229,1155,15,7
169095,1155,15:07:13,229,263,15,7
169096,263,16:00:55,12,263,16,0
169097,1234,13:16:22,60,419,13,16


In [108]:
can_used_od_trips_dataframe

,start_station_id,time,line,end_station_id,hour,minu
0,1435,16:55:43,46,1262,16,55
1,643,09:20:51,204,436,09,20
2,436,09:26:05,207,898,09,26
3,898,16:29:50,5,406,16,29
4,795,07:13:30,7,1416,07,13
...,...,...,...,...,...,...
169094,1155,15:07:03,229,1155,15,07
169095,1155,15:07:13,229,263,15,07
169096,263,16:00:55,12,263,16,00
169097,1234,13:16:22,60,419,13,16


In [122]:
#'hour','minu','line','start_station_id','end_station_id','time'
#可以获取各个站点各个时刻的上车人数，不管是不是同一个线路
od_trips_groupby_id_hour = can_used_od_trips_dataframe.groupby(['start_station_id', \
                                                                'hour']).agg({'line':list, \
                                                                              'end_station_id':list, \
                                                                              'minu':list}).reset_index()
od_trips_groupby_id_hour[od_trips_groupby_id_hour['start_station_id']==0]

,start_station_id,hour,line,end_station_id,minu
0,0,06,"[226, 226]","[345, 1177]","[38, 38]"
1,0,07,"[226, 228]","[28, 345]","[19, 35]"
2,0,08,[226],[846],[58]
3,0,09,[228],[345],[35]
4,0,10,"[226, 228, 228, 228, 228, 228, 228]","[261, 0, 1070, 1218, 128, 0, 345]","[13, 35, 35, 34, 35, 34, 34]"
5,0,11,"[226, 226, 226]","[846, 846, 507]","[50, 50, 20]"
6,0,12,[228],[467],[35]
7,0,16,[226],[0],[45]
8,0,17,[228],[345],[45]
9,0,18,"[226, 226, 226]","[846, 846, 846]","[52, 51, 52]"


In [128]:
#将各个站点的各个图像画出来，
#选择人流量大的站点，看上车人数分布
od_trips_groupby_id_hour['nums'] = od_trips_groupby_id_hour['line'].apply(lambda x:len(x))
od_trips_groupby_id_hour

,start_station_id,hour,line,end_station_id,minu,nums
0,0,06,"[226, 226]","[345, 1177]","[38, 38]",2
1,0,07,"[226, 228]","[28, 345]","[19, 35]",2
2,0,08,[226],[846],[58],1
3,0,09,[228],[345],[35],1
4,0,10,"[226, 228, 228, 228, 228, 228, 228]","[261, 0, 1070, 1218, 128, 0, 345]","[13, 35, 35, 34, 35, 34, 34]",7
...,...,...,...,...,...,...
12189,1448,13,"[1027, 256, 1027, 1027, 232]","[1166, 774, 1251, 139, 122]","[35, 27, 35, 35, 21]",5
12190,1448,14,"[232, 1027, 256, 66]","[147, 1257, 610, 1143]","[01, 33, 19, 22]",4
12191,1448,15,"[66, 232, 66]","[263, 1177, 1357]","[13, 12, 13]",3
12192,1448,16,[1027],[43],[45],1


In [150]:
# 对DataFrame进行groupby并计算每个组的总和
grouped_sum = od_trips_groupby_id_hour.groupby('start_station_id')['nums'].transform('sum')
grouped_sum
# 计算每行的值占总和的比例
od_trips_groupby_id_hour['nums_proportion'] = od_trips_groupby_id_hour['nums'] / grouped_sum
od_trips_groupby_id_hour

,start_station_id,hour,line,end_station_id,minu,nums,nums_proportion
0,0,06,"[226, 226]","[345, 1177]","[38, 38]",2,0.090909
1,0,07,"[226, 228]","[28, 345]","[19, 35]",2,0.090909
2,0,08,[226],[846],[58],1,0.045455
3,0,09,[228],[345],[35],1,0.045455
4,0,10,"[226, 228, 228, 228, 228, 228, 228]","[261, 0, 1070, 1218, 128, 0, 345]","[13, 35, 35, 34, 35, 34, 34]",7,0.318182
...,...,...,...,...,...,...,...
12189,1448,13,"[1027, 256, 1027, 1027, 232]","[1166, 774, 1251, 139, 122]","[35, 27, 35, 35, 21]",5,0.106383
12190,1448,14,"[232, 1027, 256, 66]","[147, 1257, 610, 1143]","[01, 33, 19, 22]",4,0.085106
12191,1448,15,"[66, 232, 66]","[263, 1177, 1357]","[13, 12, 13]",3,0.063830
12192,1448,16,[1027],[43],[45],1,0.021277


In [133]:
od_trips_groupby_id_4_nums = od_trips_groupby_id_hour.groupby('start_station_id').agg({'nums':list}).reset_index()
od_trips_groupby_id_4_nums

,start_station_id,nums
0,0,"[2, 2, 1, 1, 7, 3, 1, 1, 1, 3]"
1,1,"[6, 7, 6, 6, 4, 1, 3, 3, 1, 1, 1]"
2,2,"[3, 8, 25, 28, 28, 8, 15, 20, 10, 20, 19, 7, 8..."
3,4,"[1, 5, 2, 1, 1, 1, 1, 1, 1]"
4,7,"[34, 32, 34, 54, 38, 23, 27, 34, 15, 11, 27, 1..."
...,...,...
1126,1441,"[1, 1, 1, 1, 1, 1]"
1127,1442,"[2, 1, 1]"
1128,1443,"[2, 13, 7, 3, 19, 11, 4, 4, 8, 7, 11, 12, 4, 1..."
1129,1444,[1]


In [143]:
od_trips_groupby_id_4_nums['sum'] = od_trips_groupby_id_4_nums['nums'].apply(lambda x:sum(x))
#od_trips_groupby_id_4_nums
station_id_2_swip_times = list(zip(od_trips_groupby_id_4_nums['start_station_id'], \
                                   od_trips_groupby_id_4_nums['sum']))
#station_id_2_swip_times
sorted_station_id_2_swip_times = sorted(station_id_2_swip_times,key=lambda x:x[1],reverse=True)
sorted_station_id_2_swip_times

[(261, 3544),
 (759, 2193),
 (1414, 1959),
 (406, 1886),
 (349, 1807),
 (404, 1800),
 (806, 1489),
 (633, 1440),
 (795, 1405),
 (767, 1396),
 (387, 1384),
 (263, 1367),
 (1067, 1358),
 (1052, 1263),
 (610, 1193),
 (1173, 1187),
 (711, 1175),
 (1409, 1165),
 (555, 1120),
 (987, 1118),
 (1155, 1113),
 (28, 1081),
 (1011, 1075),
 (1208, 1071),
 (122, 1056),
 (553, 1023),
 (983, 993),
 (1120, 992),
 (508, 967),
 (534, 949),
 (850, 944),
 (419, 942),
 (216, 918),
 (87, 902),
 (1314, 897),
 (184, 889),
 (761, 870),
 (1358, 841),
 (447, 840),
 (1257, 840),
 (424, 812),
 (1063, 777),
 (88, 748),
 (913, 733),
 (354, 725),
 (787, 715),
 (31, 708),
 (189, 703),
 (550, 691),
 (405, 681),
 (128, 678),
 (908, 675),
 (744, 671),
 (320, 662),
 (807, 655),
 (1207, 654),
 (1056, 647),
 (758, 640),
 (899, 639),
 (1268, 633),
 (1140, 628),
 (430, 625),
 (403, 608),
 (1163, 604),
 (692, 592),
 (682, 585),
 (235, 580),
 (1009, 579),
 (1108, 575),
 (1018, 563),
 (1061, 560),
 (833, 548),
 (431, 547),
 (374, 

In [151]:
od_trips_groupby_id_hour[od_trips_groupby_id_hour['start_station_id']==261]

,start_station_id,hour,line,end_station_id,minu,nums,nums_proportion
2185,261,06,"[230, 2, 77, 248, 311, 311, 10, 230, 248, 1, 2...","[1246, 406, 144, 1157, 929, 1427, 1052, 1246, ...","[55, 19, 26, 31, 52, 38, 56, 43, 31, 52, 49, 3...",39,0.011005
2186,261,07,"[320, 27, 22, 1, 218, 72, 248, 311, 316, 316, ...","[1180, 437, 516, 406, 261, 1363, 594, 1002, 26...","[49, 58, 12, 53, 10, 25, 33, 05, 55, 55, 22, 4...",178,0.050226
2187,261,08,"[316, 320, 316, 27, 314, 320, 1, 1, 72, 77, 31...","[301, 550, 127, 1155, 908, 261, 555, 711, 261,...","[45, 56, 52, 19, 22, 27, 54, 15, 41, 23, 36, 4...",304,0.085779
2188,261,09,"[314, 33, 1, 2, 12, 314, 1, 311, 33, 2, 314, 1...","[1173, 795, 406, 216, 263, 122, 553, 1237, 117...","[37, 09, 42, 12, 32, 56, 41, 26, 28, 33, 42, 0...",482,0.136005
2189,261,10,"[22, 2, 248, 314, 311, 1, 27, 12, 314, 10, 320...","[306, 1208, 216, 1173, 60, 987, 424, 1161, 30,...","[37, 57, 48, 12, 12, 28, 17, 16, 55, 04, 34, 5...",514,0.145034
2190,261,11,"[33, 320, 66, 2, 316, 248, 72, 311, 1, 72, 314...","[1155, 1067, 1430, 349, 374, 1155, 1314, 1141,...","[14, 09, 57, 29, 40, 14, 32, 03, 06, 05, 29, 5...",415,0.117099
2191,261,12,"[248, 10, 1, 316, 316, 248, 248, 27, 1, 314, 1...","[216, 555, 711, 261, 1415, 261, 1173, 31, 1208...","[10, 56, 23, 47, 47, 09, 48, 25, 16, 34, 26, 3...",289,0.081546
2192,261,13,"[2, 1, 16, 314, 1, 248, 230, 27, 16, 33, 72, 1...","[387, 806, 633, 208, 711, 261, 1408, 552, 527,...","[15, 19, 52, 55, 22, 26, 31, 12, 21, 42, 02, 1...",260,0.073363
2193,261,14,"[248, 2, 1, 33, 27, 12, 12, 22, 2, 51, 2, 2, 7...","[767, 216, 555, 1140, 1414, 1304, 527, 87, 349...","[57, 04, 41, 14, 06, 30, 16, 41, 34, 44, 05, 2...",242,0.068284
2194,261,15,"[314, 248, 12, 2, 320, 2, 33, 33, 1, 12, 1, 1,...","[28, 787, 359, 387, 765, 263, 768, 1143, 918, ...","[35, 59, 47, 43, 32, 42, 31, 31, 51, 19, 31, 3...",204,0.057562


In [170]:
#获取每个用户第一次出发的id。
#获取可以获得的部分od对
#从这些内容估计剩余部分的od对。
#如果只有这些od对，可以看出路网内各个站点的人口流入流出情况，比如说，当他的可知od对进行完以后，这个人抵达了哪里
#可以看出各个站点在最后一程之前，各个站点的结余情况。
def process(x):
        return station_2_id[road_line_station_structure_setting \
        [x['line_identity']]['stations'][x['station_identity']]]
    except:
        return -1

day_0929_dataframe['station_id'] =  \
day_0929_dataframe.apply(lambda x: process(x)\
,axis=1)

#每个站点的刷卡数量可以通过对
day_0929_dataframe
#每个站点的总刷卡数量统计
day_0929_dataframe_groupby_station_id = \
day_0929_dataframe. \
groupby('station_id'). \
agg({'custom_precise_time':list,'line_identity':list,'card_identity':list,'car_identity':list}).reset_index()
day_0929_dataframe_groupby_station_id
day_0929_dataframe_groupby_station_id['nums'] = \
day_0929_dataframe_groupby_station_id['card_identity'].apply(lambda x:len(x))
day_0929_dataframe_groupby_station_id
day_0929_all_stations_swip_times = list(zip(day_0929_dataframe_groupby_station_id['station_id'], \
         day_0929_dataframe_groupby_station_id['nums']))
day_0929_all_stations_swip_times
sorted_day_0929_all_stations_swip_times= sorted(day_0929_all_stations_swip_times,key=lambda x:x[1],reverse=True)
sorted_day_0929_all_stations_swip_times

[(-1, 38878),
 (261, 7817),
 (759, 4303),
 (349, 4245),
 (406, 4052),
 (1414, 3963),
 (404, 3883),
 (263, 3204),
 (387, 3055),
 (633, 2952),
 (806, 2849),
 (795, 2847),
 (1067, 2837),
 (767, 2822),
 (1208, 2618),
 (1155, 2510),
 (1011, 2485),
 (1173, 2484),
 (711, 2462),
 (1052, 2439),
 (987, 2415),
 (983, 2286),
 (610, 2268),
 (1120, 2199),
 (28, 2171),
 (534, 2158),
 (1409, 2136),
 (87, 2105),
 (555, 2059),
 (122, 2056),
 (216, 1943),
 (553, 1921),
 (1314, 1910),
 (1257, 1883),
 (1268, 1820),
 (447, 1808),
 (184, 1791),
 (508, 1782),
 (761, 1733),
 (850, 1705),
 (913, 1667),
 (1358, 1643),
 (430, 1596),
 (1063, 1596),
 (424, 1553),
 (758, 1503),
 (419, 1485),
 (807, 1445),
 (403, 1375),
 (550, 1368),
 (1392, 1356),
 (744, 1344),
 (1056, 1325),
 (354, 1319),
 (908, 1318),
 (88, 1299),
 (405, 1283),
 (866, 1283),
 (128, 1230),
 (31, 1225),
 (787, 1209),
 (516, 1191),
 (320, 1172),
 (1062, 1168),
 (189, 1167),
 (431, 1164),
 (1108, 1163),
 (899, 1162),
 (1009, 1153),
 (368, 1146),
 (116

In [173]:
import folium
from folium.plugins import HeatMap

# 假设你有站点数据，每个站点包括经纬度和权重
site_data = [
    {'lat': 40.7128, 'lon': -74.0060, 'weight': 0.5},
    {'lat': 34.0522, 'lon': -118.2437, 'weight': 0.8},
    {'lat': 51.5074, 'lon': -0.1278, 'weight': 0.6}
]

# 创建地图对象
m = folium.Map(location=[site_data[0]['lat'], site_data[0]['lon']], zoom_start=5)

# 创建热力图，并添加到地图上
heat_data = [[site['lat'], site['lon'], site['weight']] for site in site_data]
HeatMap(heat_data).add_to(m)

# 保存地图为HTML文件
m.save('map.html')


'油榨街'

In [ ]:
#以各个站为起点的刷卡数量统计


In [157]:
day_0929_dataframe

,card_identity,line_identity,car_identity,station_identity,custom_day,custom_precise_time
0,0x1FDD5BE63E904A3230A0AE9AA0E40C61,28,7520,54,2023-09-29,08:59:40
1,0x1FFC5D30D96AE2C4524842A09AA67C44,84,5085,24,2023-09-29,11:08:22
2,0x65F9DAD54040FBE48972B10F3B6337DC,25,6718,4,2023-09-29,08:51:28
3,0x747A26B4727A9A3094A1881F29D01510,208,6655,1,2023-09-29,22:32:48
4,0x81A8FFB4112FDBA4782A4FB50E1C0EE0,204,4472,46,2023-09-29,19:50:37
...,...,...,...,...,...,...
377159,0x51A33F38105E35D006C03F0694252007,52,5054,22,2023-09-29,08:48:38
377160,0x5BD2D7FBE1BE24A24AFD4D3ECA27FD3F,1,4662,6,2023-09-29,10:33:45
377161,0x6D191CC173F9415C1148B434D2F9012E,203,4810,5,2023-09-29,14:57:10
377162,0x7896A07D162530E734F876D16301EFEB,73,5176,23,2023-09-29,15:08:49


In [120]:
#'hour','minu','line','start_station_id','end_station_id','time'
#可以获取各个线路各个时刻在各个站点的刷卡数量
od_trips_groupby_line_hour_id = can_used_od_trips_dataframe.groupby(['line','hour','start_station_id' \
                                                                ]).agg({\
                                                                              'end_station_id':list, \
                                                                              'minu':list}).reset_index()
od_trips_groupby_line_hour_id

,line,hour,start_station_id,end_station_id,minu
0,1,05,806,"[1409, 430]","[50, 53]"
1,1,06,128,"[1414, 349, 1392, 261, 555, 633, 261, 261, 909...","[14, 45, 36, 14, 14, 15, 56, 56, 14, 15, 56]"
2,1,06,216,"[555, 261, 431, 555]","[37, 37, 57, 37]"
3,1,06,261,"[555, 555, 555, 555, 555, 828]","[52, 52, 22, 21, 52, 43]"
4,1,06,263,"[261, 261, 261]","[19, 19, 49]"
...,...,...,...,...,...
37460,4005,16,1056,[1056],[27]
37461,4005,16,1057,[373],[59]
37462,4005,16,1389,"[1389, 765, 1389, 263]","[32, 33, 33, 33]"
37463,4600,07,985,"[28, 31]","[57, 57]"


In [31]:
day_1001_dataframe = card_info_dataframe[card_info_dataframe['custom_day']=='2023-10-01']
day_1001_dataframe

,card_identity,line_identity,car_identity,station_identity,custom_day,custom_precise_time
377164,0x0C6B84DFFBEB7E3112481AE23499566B,222,7703,1,2023-10-01,08:33:23
377165,0x13A08316FB35BC0DFE0A98D00604E7E7,6,7744,4,2023-10-01,17:51:06
377166,0x1A146FC5EC5C78862B9A10C3004FCC2C,3,7477,15,2023-10-01,15:10:36
377167,0x3DC18BE83960BE730FC7CDCB33E1F2F5,226,7053,1,2023-10-01,15:25:31
377168,0x47602A7B7A8F237606D283798F822EEC,219,7158,21,2023-10-01,14:51:54
...,...,...,...,...,...,...
724622,0x6B5CCBC949D06967421DDEFFA8682697,13,7618,22,2023-10-01,11:09:08
724623,0x7FF2BF197AE5BB5ECCBC9BC64FFEE659,7,5020,10,2023-10-01,15:34:58
724624,0x89C88A5125FCAC7607C396EDABD8FA50,320,4451,9,2023-10-01,10:13:20
724625,0xA159FB0957C071A17B4FC41C6F622556,61,3826,10,2023-10-01,17:22:23


In [149]:
#先找出每个人的od对需要做两件事：
#1.确定每个公交线路上每个上车站沿着这个线路可达的站
#2.确定每个站点与周围站点的距离。
#起点站好确定，但是终点站不好确定，看这条线路跟他下一趟车上车位置距离最近的站是哪个，这个站就是他这趟路程的终点站。
#